In [ ]:
# default_exp lightning.callbacks

In [ ]:
# export
import numpy as np
from typing import Optional
import os
import sys

import datetime
import time
from time import perf_counter
import logging

from collections import namedtuple
from tqdm.auto import tqdm

import wandb

import torch
import pytorch_lightning as pl
from pytorch_lightning.core.memory import ModelSummary, get_human_readable_count

from src.all import *

In [ ]:
import colorlog

handler = colorlog.StreamHandler()

fmt = "%(green)s[%(asctime)s %(name)s]%(reset)s: %(message)s"
colors = dict(DEBUG="purple", INFO="green", WARNING="yellow", ERROR="red", CRITICAL="red")
formatter = colorlog.ColoredFormatter(fmt=fmt, log_colors=colors, datefmt="%m/%d %H:%M:%S")
handler.setFormatter(formatter)

logging.basicConfig(format=fmt, level=logging.INFO, handlers=[handler])

In [ ]:
# export
# from : https://github.com/facebookresearch/fvcore/blob/master/fvcore/common/timer.py
class Timer:
    """
    A timer which computes the time elapsed since the start/reset of the timer.
    """

    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        """
        Reset the timer.
        """
        self._start = perf_counter()
        self._paused: Optional[float] = None
        self._total_paused = 0
        self._count_start = 1

    def pause(self) -> None:
        """
        Pause the timer.
        """
        if self._paused is not None:
            raise ValueError("Trying to pause a Timer that is already paused!")
        self._paused = perf_counter()

    def is_paused(self) -> bool:
        """
        Returns:
            bool: whether the timer is currently paused
        """
        return self._paused is not None

    def resume(self) -> None:
        """
        Resume the timer.
        """
        if self._paused is None:
            raise ValueError("Trying to resume a Timer that is not paused!")
        self._total_paused += perf_counter() - self._paused  # pyre-ignore
        self._paused = None
        self._count_start += 1

    def seconds(self) -> float:
        """
        Returns:
            (float): the total number of seconds since the start/reset of the
                timer, excluding the time when the timer is paused.
        """
        if self._paused is not None:
            end_time: float = self._paused  # type: ignore
        else:
            end_time = perf_counter()
        return end_time - self._start - self._total_paused

    def avg_seconds(self) -> float:
        """
        Returns:
            (float): the average number of seconds between every start/reset and
            pause.
        """
        return self.seconds() / self._count_start


In [ ]:
# export
class WandbImageClassificationCallback(pl.Callback):
    """ Custom callback to add some extra functionalites to the wandb logger """

    def __init__(
        self,
        num_batches: int = 16,
        log_train_batch: bool = False,
        log_preds: bool = False,
        log_conf_mat: bool = True,
    ):

        # class names for the confusion matrix
        self.class_names = list(conf_mat_idx2lbl.values())

        # counter to log training batch images
        self.num_bs = num_batches
        self.curr_epoch = 0

        self.log_train_batch = log_train_batch
        self.log_preds = log_preds
        self.log_conf_mat = log_conf_mat

        self.val_imgs, self.val_labels = None, None

    def on_train_start(self, trainer, pl_module, *args, **kwargs):
        try:
            # log model to the wandb experiment
            wandb.watch(models=pl_module.model, criterion=pl_module.loss_func)
        except:
            pass

    def on_train_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_train_batch:
            if pl_module.one_batch is None:
                pass

            else:
                one_batch = pl_module.one_batch[: self.num_bs]
                train_ims = one_batch.data.to("cpu")
                trainer.logger.experiment.log(
                    {"train_batch": [wandb.Image(x) for x in train_ims]}, commit=False
                )

    def on_validation_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_preds:
            if self.val_imgs is None and self.val_labels is None:
                self.val_imgs, self.val_labels = next(iter(pl_module.val_dataloader()))
                self.val_imgs, self.val_labels = (
                    self.val_imgs[: self.num_bs],
                    self.val_labels[: self.num_bs],
                )
                self.val_imgs = self.val_imgs.to(device=pl_module.device)

            logits = pl_module(self.val_imgs)
            preds = torch.argmax(logits, 1)
            preds = preds.data.cpu()

            ims = [
                wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                for x, pred, y in zip(self.val_imgs, preds, self.val_labels)
            ]
            log_dict = {"predictions": ims}
            wandb.log(ims, commit=False)

    def on_epoch_start(self, trainer, pl_module, *args, **kwargs):
        pl_module.val_labels_list = []
        pl_module.val_preds_list = []

    def on_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_conf_mat:
            val_preds = torch.tensor(pl_module.val_preds_list).data.cpu().numpy()
            val_labels = torch.tensor(pl_module.val_labels_list).data.cpu().numpy()
            log_dict = {
                "conf_mat": wandb.plot.confusion_matrix(
                    val_preds, val_labels, self.class_names
                )
            }
            wandb.log(log_dict, commit=False)

In [ ]:
# export
class DisableValidationBar(pl.callbacks.ProgressBar):
    "Custom Progressbar callback for Lightning Training which disables the validation bar"

    def init_sanity_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for the validation sanity run. """
        bar = tqdm(
            desc="Validation sanity check",
            position=(2 * self.process_position),
            disable=self.is_disabled,
            dynamic_ncols=True,
        )

        return bar

    def init_train_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for training. """
        bar = tqdm(
            desc="Training",
            initial=self.train_batch_idx,
            position=(2 * self.process_position),
            disable=self.is_disabled,
            dynamic_ncols=True,
        )

        return bar

    def init_validation_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for validation. """
        bar = tqdm(
            desc="Validating",
            position=(2 * self.process_position + 1),
            disable=True,
            dynamic_ncols=False,
        )

        return bar

    def init_test_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for testing. """
        bar = tqdm(
            desc="Testing",
            position=(2 * self.process_position),
            disable=self.is_disabled,
            dynamic_ncols=True,
        )

        return bar

In [ ]:
# export
class PrintLogsCallback(pl.Callback):
    "Logs Training logs to console after every epoch"
    TrainResult = namedtuple("TrainOutput", ["loss", "acc", "val_loss", "val_acc"])
    TestResult = namedtuple("TestOutput", ["test_loss", "test_acc"])

    logger = logging.getLogger("train")

    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        train_loss = metrics["train/loss_epoch"]
        train_acc = metrics["train/acc_epoch"]
        valid_loss = metrics["valid/loss"]
        valid_acc = metrics["valid/acc"]
        trn_res = self.TrainResult(
            round(train_loss.data.cpu().numpy().item(), 3),
            round(train_acc.data.cpu().numpy().item(), 3),
            round(valid_loss.data.cpu().numpy().item(), 3),
            round(valid_acc.data.cpu().numpy().item(), 3),
        )

        curr_epoch = int(trainer.current_epoch)
        self.logger.info(f"EPOCH {curr_epoch}: {trn_res}")

    def on_test_epoch_end(self, trainer, pl_module, *args, **kwargs):
        metrics = trainer.callback_metrics
        test_loss = metrics["test/loss"]
        test_acc = metrics["test/acc"]
        self.logger.info(
            f"{self.TestResult(round(test_loss.data.cpu().numpy().item(), 2), round(test_acc.data.cpu().numpy().item(), 2))}"
        )

In [ ]:
# export
class DisableProgressBar(pl.callbacks.ProgressBar):
    "Custom Progressbar callback for Lightning Training which disables the validation bar"

    def init_sanity_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for the validation sanity run. """
        bar = tqdm(
            desc="Validation sanity check",
            position=(2 * self.process_position),
            disable=True,
            dynamic_ncols=True,
        )

        return bar

    def init_train_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for training. """
        bar = tqdm(
            desc="Training",
            initial=self.train_batch_idx,
            position=(2 * self.process_position),
            disable=True,
            dynamic_ncols=True,
        )

        return bar

    def init_validation_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for validation. """
        bar = tqdm(
            desc="Validating",
            position=(2 * self.process_position + 1),
            disable=True,
            dynamic_ncols=False,
        )

        return bar

    def init_test_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for testing. """
        bar = tqdm(
            desc="Testing",
            position=(2 * self.process_position),
            disable=True,
            dynamic_ncols=True,
        )

        return bar

In [ ]:
# export
# modified from : https://github.com/facebookresearch/detectron2/blob/master/detectron2/engine/hooks.py
class IterationTimer:
    """
    Track the time spent for each iteration (each run_step call in the trainer).
    """

    def __init__(self):
        self._step_timer = Timer()
        self._total_timer = Timer()
        self._start_time = time.perf_counter()
        self.eta_list = []

    def before_train(self):
        self._start_time = time.perf_counter()
        self._total_timer.reset()
        self._total_timer.pause()

    def after_train(self, num_iter: int, stage:str = "training"):
        logger = logging.getLogger("train")
        total_time = time.perf_counter() - self._start_time
        total_time_minus_hooks = self._total_timer.seconds()
        hook_time = total_time - total_time_minus_hooks

        if num_iter > 0 and total_time_minus_hooks > 0:
            logger.info(
                "Overall {} speed: {} iterations in {} ({:.4f} s / it)".format(
                    stage,
                    num_iter,
                    str(datetime.timedelta(seconds=int(total_time_minus_hooks))),
                    total_time_minus_hooks / num_iter,
                )
            )

    def before_step(self):
        self._step_timer.reset()
        self._total_timer.resume()

    def after_step(self):
        sec = self._step_timer.seconds()
        self.eta_list.append(sec)
        self._total_timer.pause()

    def get_eta(self, max_iter: int, iteration: int):
        eta_seconds = np.median(self.eta_list[-1000:]) * (max_iter - iteration - 1)
        eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
        return eta_string

In [ ]:
# export
class ConsoleLogger(pl.Callback):
    "Fancy logger for console-logging"
    trn_res = namedtuple("TrainOutput", ["loss", "acc", "val_loss", "val_acc"])
    tst_res = namedtuple("TestOutput", ["test_loss", "test_acc"])
    curr_step = 0
    
    logger = logging.getLogger("train")

    def __init__(self, print_every: int = 50):
        self.print_every = print_every
        self.train_timer = IterationTimer()
        self.test_timer = IterationTimer()

    def on_train_start(self, trainer, pl_module, *args, **kwargs):
        self.start_time = time.perf_counter()
        
        cfg = pl_module.hparams

        model_class = str(cfg.network.transfer_learning_model._target_).split(".")[-1]
        summary = ModelSummary(pl_module)
        param_count = get_human_readable_count(summary.param_nums[0])
            
        # log information about the model
        _model = namedtuple("Model", ["base_classifier", "kind"])
        self.log_msg(f"{_model(cfg.encoder, model_class)} created, param_count: {param_count}")

        path = os.path.relpath(cfg.datamodule.im_dir)
        oof_fold = str(cfg.datamodule.curr_fold)
        trn_batches = len(pl_module.train_dataloader())
        val_batches = len(pl_module.val_dataloader())
        tst_batches = len(pl_module.test_dataloader())
        dl = pl_module.train_dataloader()
        try:
            _trn_tfms = list(dl.dataset.tfms.transforms)
        except:
            _trn_tfms = list(dl.dataset.transforms.transforms)
            
        # log information about the dataset
        self.log_msg(f"Loaded dataset from {path}, using {oof_fold} fold as the OOF validation dataset.")
        self.log_msg(f"Loaded dataset has {trn_batches} train + {val_batches} valid + {tst_batches} test.")
        self.log_msg(f"Transformations used in training: {_trn_tfms}.")
        self.log_msg(f"Images are resized to {(cfg.image_dims, cfg.image_dims, 3)}.")
        self.log_msg(f"Creating batches of size {str(cfg.datamodule.bs)} and concating the images.")

        lr_scheduler = str(cfg.scheduler.function._target_).split(".")[-1]
        optimizer = str(cfg.optimizer._target_).split(".")[-1]
        # log information about the training parameters
            
        self.log_msg(f"Scheduled epochs: {trainer.max_epochs}")
        self.log_msg(f"Accumulate batches: {trainer.accumulate_grad_batches}")
            
        self.log_msg(f"Training with {optimizer}, lrs={str(pl_module.lr_list)}, wd={str(cfg.optimizer.weight_decay)}")
        self.log_msg(f"Scheduling learining_rate via {lr_scheduler} scheduler.")
            
        self.log_msg(f"Loss Function used : {pl_module.loss_func}")
        
        if pl_module.mix_fn is not None:
            self.log_msg(f"Training with {pl_module.mix_fn}")

        # start the training job
        self.log_msg(f"Start TRAIN / VALIDATION from epoch {trainer.current_epoch}")

        self.total_iters = trn_batches * trainer.max_epochs
        self.current_iteration = 0
        self.train_timer.before_train()
        self.total_test_iters = tst_batches

    def on_train_epoch_start(self, *args, **kwargs):
        self.step_loss = 0
        self.step_acc = 0
        self.seen_batches = 0

    def on_train_batch_start(self, *args, **kwargs):
        self.train_timer.before_step()
        self.seen_batches += 1

    def on_train_batch_end(self, trainer, pl_module, *args, **kwargs):
        msg = "eta: {} iteration: {} loss: {:.3f} accuracy: {:.3f} lrs: {}"

        mini_batch_size = pl_module.hparams.datamodule.bs

        step_metrics = trainer.callback_metrics

        self.step_loss += step_metrics["train/loss_step"]
        self.step_acc += step_metrics["train/acc_step"]

        self.train_timer.after_step()

        if self.curr_step % self.print_every == 0:
            # compute average loss/accuracy
            avg_loss = self.step_loss / self.seen_batches
            avg_acc = self.step_acc / self.seen_batches

            # the learning-rates of the model parameters
            lrs = tuple(trainer.lr_schedulers[0]["scheduler"].get_lr())
            lrs = tuple([float("{0:.1e}".format(v)) for v in lrs])

            # get eta from timer
            eta = self.train_timer.get_eta(max_iter=self.total_iters, iteration=self.current_iteration)

            self.log_msg(msg.format(eta, self.current_iteration, avg_loss, avg_acc, lrs))

        # increment iterations
        self.current_iteration += 1
        self.curr_step += 1

    def on_epoch_end(self, trainer, pl_module, *args, **kwargs):
        metrics = trainer.callback_metrics

        train_loss = metrics["train/loss_epoch"]
        train_acc = metrics["train/acc_epoch"]

        valid_loss = metrics["valid/loss"]
        valid_acc = metrics["valid/acc"]

        _res = self.trn_res(
            round(train_loss.data.cpu().numpy().item(), 3),
            round(train_acc.data.cpu().numpy().item(), 3),
            round(valid_loss.data.cpu().numpy().item(), 3),
            round(valid_acc.data.cpu().numpy().item(), 3),
        )

        curr_epoch = int(trainer.current_epoch)
        self.log_msg(f"EPOCH {curr_epoch}: (100 % done) {_res}")

    def on_train_end(self, *args, **kwargs):
        time_elasped = time.perf_counter() - self.start_time
        self.log_msg(f"Total training time: {str(datetime.timedelta(seconds=int(time_elasped)))}")
        self.train_timer.after_train(num_iter=self.current_iteration)

    def on_test_start(self, trainer, pl_module, *args, **kwargs):
        self.test_iters = 0
        self.test_start = time.perf_counter()
        self.test_timer.before_train()

        path = os.path.relpath(trainer.checkpoint_callback.dirpath)
        self.log_msg(f"Start TEST on {self.total_test_iters} batches")
        
        dl = pl_module.val_dataloader()
        try:
            _val_tfms = list(dl.dataset.tfms.transforms)
        except:
            _val_tfms = list(dl.dataset.transforms.transforms)
        self.log_msg(f"Transformations used: {_val_tfms}")
        
    def on_test_batch_start(self, trainer, pl_module, *args, **kwargs):
        self.test_timer.before_step()

    def on_test_batch_end(self, trainer, pl_module, *args, **kwargs):
        self.test_timer.after_step()

        if self.test_iters % self.print_every == 0:
            eta = self.test_timer.get_eta(max_iter=self.total_test_iters, iteration=self.test_iters)
            self.log_msg(f"Testing done {self.test_iters}/{self.total_test_iters}. ETA={eta}")
        self.test_iters += 1

    def on_test_epoch_end(self, trainer, pl_module, *args, **kwargs):
        metrics = trainer.callback_metrics
        test_loss = metrics["test/loss"]
        test_acc = metrics["test/acc"]

        loss = round(test_loss.data.cpu().numpy().item(), 2)
        accuracy = round(test_acc.data.cpu().numpy().item(), 2)
        self.log_msg(f"Test results: (100 % done) {self.tst_res(loss, accuracy)}")

    def on_test_end(self, *args, **kwargs):
        time_elasped = time.perf_counter() - self.test_start
        self.log_msg(f"Total testing time: {str(datetime.timedelta(seconds=int(time_elasped)))}")
        self.test_timer.after_train(num_iter=self.test_iters, stage="testing")

    def log_msg(self, msg: str):
        self.logger.info(msg)

    def log_line(self):
        self.logger.info("-" * 80)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from torch import nn
from src.networks import *

from hydra.experimental import compose, initialize
from hydra.utils import instantiate
from omegaconf import OmegaConf

In [ ]:
overrides = [
    "image_dims=120",
    "datamodule.bs=5",
    "datamodule.num_workers=0",
    "general=default",
    "trainer=fast-dev-cpu",
    "mixmethod=mixup",
    "network=transferlearning",
    "augmentations=custom-augs",
    "mixmethod.steps=2",
    "network.activation=mish",
]

with initialize(config_path=os.path.relpath("../conf/")):
    cfg = compose(config_name="effnet-base", overrides=overrides)

In [ ]:
cfg.image_dims = 255

trn_augs = None
val_augs = None

dm = instantiate(
    cfg.datamodule, train_augs=trn_augs, valid_augs=val_augs, default_config=cfg
)

activation_func = activation_map[cfg.network.activation]
logging.info(f"Using {activation_func()} activation function.")
encoder = timm.create_model(cfg.encoder, pretrained=False, act_layer=activation_func)
model = TransferLearningModel(encoder, cut=-2, c=5, act=activation_func(inplace=True))
model = LightningCassava(model=model, conf=cfg)

lg = ConsoleLogger(print_every=5)

dm.prepare_data()
dm.setup()

trainer = instantiate(
    cfg.trainer,
    callbacks=[DisableProgressBar(), lg],
    max_epochs=2,
    limit_train_batches=10,
    limit_val_batches=2,
    limit_test_batches=5,
    weights_summary=None,
)

[01/25 01:54:20 root]: Using Mish() activation function.
GPU available: False, used: False
[01/25 01:54:22 lightning]: GPU available: False, used: False
TPU available: False, using: 0 TPU cores
[01/25 01:54:22 lightning]: TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, datamodule=dm)

[01/25 01:54:23 train]: Model(base_classifier='tf_efficientnet_b0_ns', kind='TransferLearningModel') created, param_count: 5.3 M
[01/25 01:54:23 train]: Loaded dataset from ../../Datasets/cassava/train_images, using 0 fold as the OOF validation dataset.
[01/25 01:54:23 train]: Loaded dataset has 3424 train + 856 valid + 856 test.
[01/25 01:54:23 train]: Transformations used in training: [RandomResizedCropAndInterpolation(size=(255, 255), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR PIL.Image.BICUBIC), RandomHorizontalFlip(p=0.5), RandomVerticalFlip(p=0.5), ColorJitter(brightness=[0.9, 1.1], contrast=[0.9, 1.1], saturation=[0.9, 1.1], hue=None), ToTensor(), Normalize(mean=tensor([0.4298, 0.4962, 0.3130]), std=tensor([0.2142, 0.2191, 0.1954])), <timm.data.random_erasing.RandomErasing object at 0x7fe3fcffcdd0>].
[01/25 01:54:23 train]: Images are resized to (255, 255, 3).
[01/25 01:54:23 train]: Creating batches of size 5 and concating the images.
[01/25 01:54

1

In [ ]:
_ = trainer.test(datamodule=dm, verbose=False)

[01/25 01:55:06 train]: Start TEST on 856 batches
[01/25 01:55:06 train]: Transformations used: [Resize(size=291, interpolation=PIL.Image.BILINEAR), CenterCrop(size=(255, 255)), ToTensor(), Normalize(mean=tensor([0.4298, 0.4962, 0.3130]), std=tensor([0.2142, 0.2191, 0.1954]))]
[01/25 01:55:06 train]: Testing done 0/856. ETA=0:06:35
[01/25 01:55:09 train]: Test results: (100 % done) TestOutput(test_loss=0.75, test_acc=0.04)
[01/25 01:55:09 train]: Total testing time: 0:00:02
[01/25 01:55:09 train]: Overall testing speed: 5 iterations in 0:00:02 (0.4672 s / it)


In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03_layers.ipynb.
Converted 03a_networks.ipynb.
Converted 04_optimizers_schedules.ipynb.
Converted 05_lightning.data.ipynb.
Converted 05a_lightning.core.ipynb.
Converted 05b_lightning.callbacks.ipynb.
Converted 06_fastai.core.ipynb.
Converted index.ipynb.
